In [147]:
import pandas as pd
import pickle
import numpy as np
import datetime

In [157]:
def get_data(coin):
    data = pd.read_csv('newdata/{}.csv'.format(coin.upper()))
    data['date'] = data.date.apply(datetime.datetime.fromtimestamp)
    data.set_index('date', inplace=True)
    data = data
    return data.sort_index().tail(11)

def make_all_features(data):
    price = ['open','close','high','low']
    data['price'] = data[price].mean(axis=1)
    data['price_change'] = data.price.pct_change()
    data['spread']  = data.high = data.low
    data.drop(price,axis=1,inplace=True)
    data['sma_5'] = data['price'].rolling(5).mean()
    data['sma_10'] = data['price'].rolling(10).mean()
    data['mean_spread3'] = data['spread'].rolling(3).mean()
    data['mean_spread6'] = data['spread'].rolling(5).mean()
    data['mean_quote_volume3'] = data['quoteVolume'].rolling(3).mean()
    data['mean_quote_volume5'] = data['quoteVolume'].rolling(5).mean()
    data['pc_ch_5'] = data.sma_5.pct_change()
    data['pc_ch_10'] = data.sma_10.pct_change()
    data['sma5_ask_diff'] = (data.sma_5  - data.price)
    data['sma10_ask_diff'] = (data.sma_10  - data.price)
    data.dropna(inplace=True)
    return data[['quoteVolume', 'volume', 'weightedAverage', 'price_change', 'spread',
       'mean_spread3', 'mean_spread6', 'mean_quote_volume3',
       'mean_quote_volume5', 'pc_ch_5', 'pc_ch_10', 'sma5_ask_diff',
       'sma10_ask_diff']].values


def feature_eng(x):
    with open('scalers/{}'.format(coin), 'rb') as f:
        ss = pickle.load(f)
    return ss.transform(x)

def predict(x):
    with open('models/{}'.format(coin), 'rb') as f:
        model = pickle.load(f)
    return model.predict(x), model.predict_proba(x)

if __name__ == '__main__':
    pred_map = {-1:'down',0:'no_movement',1:'up'}
    coin = input('the coin name >>> ')
    d = get_data(coin)
    x = make_all_features(d)
    x = feature_eng(x)
    pred ,proba = predict(x)
    print('this predection is {}'.format(pred_map[pred[0]]))
    print('probability of down is {} || no movement is {}|| up is {}'.format(proba[:,0],proba[:,1],proba[:,2]))

the coin name >>> xem
this predection is down
probability of down is [ 0.93] || no movement is [ 0.01]|| up is [ 0.06]
